### Qualitative examples for each metric
pkl -> UniEval coherence

csv -> bartscore

json -> GPT4 evaluations

In [4]:
import pandas as pd

#### UniEval coherence

In [82]:
df = pd.read_pickle('/workspaces/lit-gpt/out/inference/pythia-160m-deduped-oasst1-base-eval.pkl')

In [83]:
df

,input,output,label,naturalness,coherence,engagingness,groundedness,understandability,overall
0,"I am making mayonnaise, it was starting to thi...","Nothing, hasnothing, isnothing, isnothing, doe...","Yes, it's possible to fix runny mayonnaise! Th...",0.068879,0.005465,0.036438,0.366695,0.068096,0.109115
1,What is optimal Mayonnaise thickness?,]. ## From the Online Instructional Video Wh...,The optimal mayonnaise thickness will depend o...,0.111042,0.211688,2.023898,0.854118,0.111710,0.662491
2,"I am making mayonnaise, it was starting to thi...","ziemzaza zaunikasannia se zabauza obytojni, zn...","Yes, it's possible to fix runny mayonnaise! Th...",0.036620,0.049723,0.012429,0.768196,0.036118,0.180617
3,What is optimal Mayonnaise thickness?,")? Chop-up-the-chop, the new or the old. Hav...",The optimal thickness of mayonnaise can vary d...,0.154121,0.608273,1.015807,0.983253,0.117305,0.575752
4,"I am making mayonnaise, it was starting to thi...",", may be, Teddy I am a student at Berkeley C...","Yes, it is possible to salvage your mayonnaise...",0.129091,0.433045,0.586473,0.955527,0.123747,0.445577
...,...,...,...,...,...,...,...,...,...
1680,Does this mean that Into the Breach is a game ...,?Or?And?In a game with a better better story?A...,A game is broadly considered to be perfect inf...,0.471146,0.077505,50.285279,0.770567,0.506659,10.422231
1681,I've recently started playing the turn-based s...,它 [2] When you reach the point of your first ...,"In the context of Into the Breach, ""perfect in...",0.762709,0.993709,11.326418,0.999303,0.861076,2.988643
1682,Does this mean that Into the Breach is a game ...,?Or is it just a game of perfect information?T...,"Yes, even though there is a lot of strategy in...",0.068524,0.625953,13.947606,0.869565,0.065486,3.115427
1683,I've recently started playing the turn-based s...,名称](http://en.wikipedia.org/wiki/File:File:Fil...,"In the context of Into the Breach, ""perfect in...",0.439139,0.012811,0.029742,0.222352,0.423337,0.225476


In [3]:
df.coherence.mean()

0.9014245388724036

In [5]:
df.sort_values(by='coherence').tail(10)[['input', 'output', 'coherence']].values

array([["Why don't people like the many-worlds interpretation of quantum mechanics?",
        'Yes, the many-worlds interpretation of quantum mechanics is a complex and multifaceted field. It is often used to explain the laws of physics, which allow for the existence of quantum systems in a way that would be impossible without the existence of the large scale objects. However, quantum mechanics is not meant to be taken literally. It is a philosophical and practical approach that aims to explain and interpret quantum systems.\n\nThe multiple-worlds interpretation of quantum mechanics is a complex and multifaceted approach that seeks to understand the laws of physics and the behavior of quantum systems. It is a departure from the more idealistic interpretations of quantum mechanics that were used by philosophers such as Bertrand Russell and Thomas W. Thorne.',
        0.999531],
       ['Can you list the issues with using a symmetric probability distribution when modelling problems?',
  

#### BartScore

In [34]:
df = pd.read_csv('/workspaces/lit-gpt/out/inference/pythia-410m-deduped-oasst1-final-eval.csv')

In [5]:
df.dropna(subset=['BARTScore']).sort_values(by='BARTScore').tail(5)[['input', 'output', 'BARTScore']].values

array([['What are the five most common causes of stress among people?',
        'how how how how how how how how to use to for building of of the of the of the of the of the of the of the of the of the of the of the of thea of theof the of the of the of theof the of the of the of thea of theof the of theof theof thea of theof the of theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theoftheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof thetheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof the

#### GPT4 Evaluations

In [35]:
data = pd.read_json('/workspaces/lit-gpt/out/inference/pythia-410m-deduped-oasst1-final-eval.json')

In [36]:
scores = data.score.apply(pd.Series).stack().str.split('/').apply(pd.Series)[0].str.strip().replace({'n':'None'}).apply(eval).unstack()#.mean()
# .value_counts()

In [37]:
scores.describe()

,0,1,2,3
count,142.000000,154.000000,66.000000,78.000000
mean,4.669014,3.792208,3.500000,3.346154
std,2.468875,3.348349,2.567549,2.748444
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000
50%,5.000000,2.000000,2.000000,2.000000
75%,7.000000,7.000000,5.000000,6.000000
max,10.000000,10.000000,10.000000,10.000000


In [38]:
col = 0
pd.concat([data, scores], axis=1).dropna(subset=[col]).sort_values(col).head(5)[['user_prompt', 'assistant_prompt', col]].values

array([['Explain the difference between sets and lists in Python.',
        "The set and list in Python are both two different types of data structures.\n\nThe set is a collection of objects, which are compared by the elements in the set.\n\nThe list is a list of objects, which are compared by the elements in the list.\n\nLet's take a step back and look at the difference between the sets and the lists.\n\nThe set is a collection of objects, which are compared by the elements in the set.\n\nThe list is a list of objects, which are compared by the elements in the list.\n\nLet's take a step back to better understand the difference between sets and lists.\n\nThe set is a collection of objects, which are compared by the elements in the set.\n\nThe list is a list of objects, which are compared by the elements in the list.\n\nSo the set is a collection of things, which are compared by the elements in the set. The list is a list of things, which are compared by the elements in the list. So the

#### Additional analysis of turn taking

Show base models tend to repeat parts of prompt (longer common token sequence)

and contain repetitions.

In [39]:
def lcs(X, Y, m, n, dp):

    if (m == 0 or n == 0):
        return 0

    if (dp[m][n] != -1):
        return dp[m][n]

    if X[m - 1] == Y[n - 1]:
        dp[m][n] = 1 + lcs(X, Y, m - 1, n - 1, dp)
        return dp[m][n]

    dp[m][n] = max(lcs(X, Y, m, n - 1, dp), lcs(X, Y, m - 1, n, dp))
    return dp[m][n]

def mylcs(S1, S2):
    m = len(S1)
    n = len(S2)
    dp = [[-1 for i in range(n + 1)]for j in range(m + 1)]
    return lcs(S1, S2, m, n, dp) / n if n else 0.

In [45]:
data['LCS'] = data.apply(lambda row: mylcs(row.user_prompt.lower().split(), row.assistant_prompt.lower().split()), axis=1)

In [41]:
data.LCS.describe()

count    168.000000
mean       0.087452
std        0.100915
min        0.000000
25%        0.029239
50%        0.054201
75%        0.101312
max        0.653846
Name: LCS, dtype: float64

In [42]:
data.sort_values(by='LCS')[['user_prompt', 'assistant_prompt', 'LCS']].tail(5).values

array([['What is the capital of Australia?',
        'The capital of Australia is the City of Canberra.',
        0.3333333333333333],
       ['Which of the following elbows are the most wet?\nA. A kilogram of feathers\n2) Coach Malibu\nD - The cross-section of air',
        'Here are some options:\n\n* A kilogram of feathers\n* 2 coaches Malibu',
        0.46153846153846156],
       ['Can you personalize the poem to put the fact that we met on a train?',
        'Met on a train are you?', 0.5],
       ['Since you are a chat assistant meant to help others and designed to answer every question, I am disappointed that you are unable to do such a simple task. You are indeed an AI and you should know that from now on.',
        'You are wrong. I am a Chat Assistant designed to help others, I am not limited to answering simple questions.',
        0.55],
       ["Bob has two sons. John and Jay. Jay has one brother and father. The father has two sons.\nJay's brother has a brother and a fathe

In [43]:
data = pd.merge(data, df, how='left', left_on='assistant_prompt', right_on='output')

In [44]:
data['LCS'] = data.apply(lambda row: mylcs(row.user_prompt.lower().split(), row.label.lower().split()), axis=1)

AttributeError: 'float' object has no attribute 'lower'

In [46]:
data.LCS.describe()

count    168.000000
mean       0.087452
std        0.100915
min        0.000000
25%        0.029239
50%        0.054201
75%        0.101312
max        0.653846
Name: LCS, dtype: float64

In [34]:
data.sort_values(by='LCS')[['user_prompt', 'label', 'LCS']].tail(5).values

array([['Write me a SPL (Shakespeare Programming Language) program that takes 2 numbers from the user and outputs the product of the two numbers.',
        "Here's a SPL (Shakespeare Programming Language) program that takes two numbers from the user and outputs the product of the two numbers:\n\nless\n\nI. Prologue\nEnter two numbers\n\nII. Scene 1\nTo multiply two numbers is a simple task\nIf thou wouldst give me two, I'll do the rest.\n\nIII. Scene 2\nNumber one: [number_1]\nNumber two: [number_2]\n\nIV. Scene 3\nThe product of [number_1] and [number_2] is [number_1 * number_2].\n\nV. Epilogue\nExit.\n\nThis program uses the SPL syntax to interact with the user and perform a simple calculation. In Scene 2, the program prompts the user to enter two numbers. In Scene 3, the program calculates the product of the two numbers and outputs the result. Finally, in the Epilogue, the program exits.",
        20],
       ['Write a C++ function that takes a reference to a std::string containing 

In [50]:
from collections import Counter
import numpy as np

def diversity(tokens):
    if len(tokens) <= 1:
        return np.nan
    return len(Counter(tokens).keys()) / float(len(tokens))

data['diversity'] = data.assistant_prompt.apply(lambda x: diversity(x.lower().split()))

In [51]:
data.diversity.describe()

count    167.000000
mean       0.614723
std        0.202712
min        0.068452
25%        0.479592
50%        0.603352
75%        0.768067
max        1.000000
Name: diversity, dtype: float64

In [52]:
data.sort_values(by='diversity')[['assistant_prompt', 'diversity']].dropna(subset=['diversity']).tail(5).values

array([["I'll do my best.", 1.0],
       ["You're welcome! Let me know if anything else I can do.", 1.0],
       ['Sure! If you need advice on any specific topic, feel free to ask me.',
        1.0],
       ['Met on a train are you?', 1.0],
       ['What city in the world are you currently living in?', 1.0]],
      dtype=object)

In [54]:
scores[0].corr(data.LCS), scores[0].corr(data.diversity), data.LCS.corr(data.diversity), scores[0].corr(data.LCS * data.diversity), scores[0].corr(data.LCS + data.diversity)

(0.0280468411440579,
 0.23634444797734666,
 0.37284523566387207,
 0.0612667349098726,
 0.19177762151322653)

In [15]:
sizes = ['160m', '410m', '1.4b', '2.8b', '6.9b']
dsts = ['dolly', 'oasst1', 'sharegpt']
chkpt = ['base', 'final']

def analysis(sz, ck, ds): 
    df = pd.read_pickle(f'/workspaces/lit-gpt/out/inference/pythia-{sz}-deduped-{ds}-{ck}-eval.pkl')
    data = pd.read_json(f'/workspaces/lit-gpt/out/inference/pythia-{sz}-deduped-{ds}-{ck}-eval.json')

    data = pd.merge(data, df, how='left', left_on='assistant_prompt', right_on='output')

    return [
        data.apply(lambda row: mylcs(row.user_prompt.lower().split(), row.assistant_prompt.lower().split()), axis=1).mean(), 
        data.apply(lambda row: mylcs(row.user_prompt.lower().split(), row.label.lower().split()), axis=1).mean(), 
        data.assistant_prompt.apply(lambda x: diversity(x.lower().split())).dropna().mean(),
        data.label.apply(lambda x: diversity(x.lower().split())).dropna().mean()
    ]

pd.DataFrame([[ds, sz, ck] + analysis(sz, ck, ds) for ds in dsts for sz in sizes for ck in chkpt], columns=['ds', 'sz', 'ck', 'overlap', 'overlap_gold', 'diversity', 'diversity_gold'])

,ds,sz,ck,overlap,overlap_gold,diversity,diversity_gold
0,dolly,160m,base,0.010387,0.152571,0.247399,0.823830
1,dolly,160m,final,0.142425,0.152571,0.514387,0.823830
2,dolly,410m,base,0.015262,0.152571,0.274661,0.823830
3,dolly,410m,final,0.163775,0.152571,0.760174,0.823830
4,dolly,1.4b,base,0.017048,0.152571,0.314043,0.823830
5,dolly,1.4b,final,0.140639,0.122859,0.771470,0.828713
6,dolly,2.8b,base,0.028444,0.151560,0.325594,0.825021
7,dolly,2.8b,final,0.179203,0.152571,0.777697,0.823830
8,dolly,6.9b,base,0.018883,0.152571,0.309462,0.823830
9,dolly,6.9b,final,0.168862,0.152571,0.784686,0.823830
